<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/openai/TikToken_count_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tiktoken

https://github.com/openai/tiktoken

# OpenAi Pricing
https://openai.com/api/pricing/


# How OpenAi Count tokens
https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them

https://platform.openai.com/tokenizer


In [1]:
!pip install tiktoken openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [2]:
import tiktoken
from openai import OpenAI

In [3]:
from google.colab import userdata
api_key = userdata.get('KEY_OPENAI')

In [8]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are a fantastic coker"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Puedes decirme como prepara bakalado a la Vizcaina?"
        }
      ]
    }

  ],
  temperature=1,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [45]:
response.choices[0].message

ChatCompletionMessage(content='¡Por supuesto! El bacalao a la vizcaína es un plato tradicional de la cocina vasca, muy sabroso y lleno de historia. Aquí tienes una receta básica para prepararlo:\n\n### Ingredientes:\n- 1 kg de bacalao desalado\n- 3 cebollas\n- 3 dientes de ajo\n- 2-3 pimientos choriceros secos\n- 400 g tomates maduros o 1 lata de tomates triturados\n- 1 hoja de laurel\n- Aceite de oliva virgen extra\n- Sal\n- Pimienta\n- Agua\n\n### Instrucciones:\n\n1. **Desalar el bacalao:**\n   - Si tienes bacalao seco, deberás desalarlo antes de cocinarlo. Para ello, colócalo en un recipiente con agua fría y cámbiala cada 8 horas durante al menos 48 horas. Guarda el recipiente en el refrigerador para evitar que se dañe.\n\n2. **Preparar los pimientos choriceros:**\n   - Remoja los pimientos choriceros en agua caliente durante al menos 2 horas o hasta que estén blandos. Luego, ábrelos, retira las semillas y raspa la carne con una cuchara. Reserva esta carne de pimiento.\n\n3. **Hace

In [9]:
response.usage

CompletionUsage(completion_tokens=512, prompt_tokens=30, total_tokens=542)

In [10]:
(response.usage.prompt_tokens * 5) /10**6 +   (response.usage.completion_tokens * 15) /10**6

0.00783

In [11]:
# enc = tiktoken.get_encoding("cl100k_base")
enc = tiktoken.encoding_for_model("gpt-4o")

In [41]:
m = """role system text: "You are a fantastic coker",role user text: "Puedes decirme como prepara bakalado a la Vizcaina?"
"""
len(enc.encode(m))

30

## Counting Tokens In Text

In [15]:
book_text = """
Mrs. Darling quivered and went to the window. It was securely fastened.
She looked out, and the night was peppered with stars. They were
crowding round the house, as if curious to see what was to take place
there, but she did not notice this, nor that one or two of the smaller
ones winked at her. Yet a nameless fear clutched at her heart and made
her cry, “Oh, how I wish that I wasn’t going to a party to-night!”

Even Michael, already half asleep, knew that she was perturbed, and he
asked, “Can anything harm us, mother, after the night-lights are lit?”

“Nothing, precious,” she said; “they are the eyes a mother leaves
behind her to guard her children.”

She went from bed to bed singing enchantments over them, and little
Michael flung his arms round her. “Mother,” he cried, “I’m glad of
you.” They were the last words she was to hear from him for a long
time.
"""

In [25]:
enc.encode(book_text)[:10]

[198, 91702, 13, 165836, 474, 62076, 326, 5981, 316, 290]

In [16]:
len(enc.encode(book_text))

222

In [17]:
def num_tokens_from_string(string: str, model_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [18]:
num_tokens_from_string(book_text, "gpt-3.5-turbo")

224

In [19]:
num_tokens_from_string(book_text, "text-davinci-003")

253

## Counting Tokens in Messages (for Chat API)

In [20]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [21]:
example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]


In [22]:
num_tokens_from_messages(example_messages, "gpt-3.5-turbo")

127

In [23]:
num_tokens_from_messages(example_messages, "gpt-4")

129